In [1]:
import pandas as pd

# You need to change 3 parts for each gesture:
# Frames File path, Elan file path and name of the final file

In [2]:


def make_csv_with_ground_truth(frames_file_path, elan_annotation_file_path, new_file_path):
    frames = pd.read_csv(frames_file_path)

# set timestamp column as index, so we can do timerange based selections
    frames["timestamp"] = pd.to_timedelta(frames["timestamp"], unit="ms")
    frames = frames.set_index("timestamp")
    frames.index = frames.index.rename("timestamp")


# the default label will be 'idle'
    frames["ground_truth"] = "idle"

# read the ELAN file as CSV; only read the relevant columns (specified by `usecols`)
    annotations = pd.read_csv(elan_annotation_file_path, sep="\t", header=None, usecols=[3, 5, 8], names=["start", "end", "label"])

# convert start and end timestamps into a datetime-datatype – this is important for the next step
    annotations["start"] = pd.to_timedelta(annotations["start"], unit="s")
    annotations["end"]   = pd.to_timedelta(annotations["end"], unit="s")

    for idx, ann in annotations.iterrows():
        annotated_frames = (frames.index >= ann["start"]) & (frames.index <= ann["end"])
        frames.loc[annotated_frames, "ground_truth"] = ann["label"]


    frames.index = frames.index.astype(int) // 1_000_000  # convert index back to integers, so it looks nicer in CSV

# save all data with the new ground_truth column into a new file:
    frames.to_csv(new_file_path, index=True)



In [ ]:
video_names = ["down", "flip table", "pinch", "rotate", "rotate2", "spin", "spread", "swipe", "swipe2", "up", "idle"]

for name in video_names:
    frames_file_path = f"C:/AAA Sophia/Uni/Semester 7/ML/csv_results/{name}.csv"
    elan_annotation_file_path = f"C:/AAA Sophia/Uni/Semester 7/ML/Videos Gestenerkennung ML/{name}.txt"
    new_file_path = f"C:/AAA Sophia/Uni/Semester 7/ML/csv_with_ground_truth/{name}_with_ground_truth.csv"
    make_csv_with_ground_truth(frames_file_path, elan_annotation_file_path, new_file_path)